In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Définition de l'User-Agent et des en-têtes
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.jumia.com.ng/',
    'Connection': 'keep-alive'
}

# Liste des URLs des sites à scraper
urls = [
    'https://www.jumia.com.ng/catalog/?q=smartwatch',
    'https://www.jumia.com.ng/catalog/?q=smartphone',
    'https://www.jumia.com.ng/catalog/?q=airpods',
    'https://www.jumia.com.ng/catalog/?q=laptops'
]

# Stockage des résultats
all_data = []

# Fonction pour scraper une URL donnée
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=1000)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Détection du site cible
        if "jumia.com.ng" in url:
            site = "jumia"
            items = soup.select('#jm > main > div.aim.row.-pbm > div.-pvs.col12 > section > div > article')

            for item in items:
                # Extraction des données
                title = item.select_one('h3')
                price = item.select_one('div.prc')
                rating = item.select_one('div.rev > div')
                reviews = item.select_one('div.rev')
                discount = item.select_one('div.bdg._dsct._sm') 
                # Nettoyage des données
                title_text = title.text.strip() if title else "N/A"
                price_text = price.text.strip() if price else "N/A"
                rating_text = rating.text.strip() if rating else "N/A"
                reviews_text = reviews.text.strip() if reviews else "0"
                stock_text = "Non renseigné"
                discount_text = discount.text.strip() if discount else "Aucun rabais"  


                # Ajout des données à la liste
                if title_text and price_text:
                    all_data.append({
                        'Site': site,
                        'Titre': title_text,
                        'Prix (Naira)': price_text,
                        'Note': rating_text.replace('out of 5', ' '),
                        'Avis': reviews_text,
                        'Stock': stock_text,
                        'Rabais': discount_text  # Ajout du rabais
                    })
        else:
            print(f"⚠️ Site non pris en charge : {url}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête {url}: {e}")

    time.sleep(2)  # Pause pour éviter le blocage

# Scraping de toutes les URLs
for url in urls:
    scrape_url(url)

# Conversion des résultats en DataFrame Pandas
df = pd.DataFrame(all_data)

# Affichage du DataFrame
print(df)

# Sauvegarde en CSV
if not df.empty:
    df.to_csv("resultats_scraping_jumiaNG1.csv", index=False, encoding='utf-8-sig')
    print("\n✅ Données sauvegardées dans 'resultats_scraping_jumiaNG.csv'")
else:
    print("\n⚠️ Aucune donnée collectée.")

      Site                                              Titre Prix (Naira)  \
0    jumia  TiLECC T800 Smart Watch Ultra 8 Door Access  S...     ₦ 10,599   
1    jumia  TiLECC T800 Smart Watch Ultra 8 Door Access  S...     ₦ 10,999   
2    jumia  Ultra Smartwatches 8 Men Ultra Women IP67 Wate...     ₦ 13,528   
3    jumia  Oraimo Smart Watch Magnetic Charger Cable & Or...      ₦ 2,799   
4    jumia  2023 Smart Watch Ultra Series 8 NFC Smartwatch...     ₦ 13,752   
..     ...                                                ...          ...   
155  jumia  Hp EliteBook 840 G5 Intel Core I5-16GB RAM-256...    ₦ 475,000   
156  jumia  Hp CHROMEBOOK  INTEL CELERON ,4GB RAM,16GB WIN...    ₦ 130,000   
157  jumia  DELL Latitude 7490 Intel Core I5- 16GB RAM/512...    ₦ 450,000   
158  jumia  DELL Latitude 7490 Intel Core I5- 16GB RAM/512...    ₦ 440,000   
159  jumia  Hp FLEXIBLE USB LIGHT FOR COMPUTERS,LAPTOP & P...      ₦ 2,500   

      Note                Avis          Stock        Rabais  
0